In [1]:
# Импортируем все необходимые библиотеки и задаем сид для рандомизатора
import pandas as pd
import numpy as np
import nltk
import pickle

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from nltk.classify.scikitlearn import SklearnClassifier
from nltk.tokenize import RegexpTokenizer
from sklearn.preprocessing import LabelEncoder

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

np.random.seed(1)

In [7]:
# Считываем данные на которых будем обучать
data = pd.read_csv(r"data\reviews.csv", index_col=0)
data

,review,assessment
0,"Игра, в которой победить может даже злодей. От...",True
1,Обалденная игра для компании. Помогает разнооб...,True
2,"Весёлая настольная игра для компании, которая ...",True
3,Общие впечатления Игра огонь даже для взрослог...,True
4,Случайно наткнулся на эту игрушку в детском ми...,True
...,...,...
96,Ерунда полная! Половина больших бакуганов не т...,False
97,"Ужасное качество фишек , древесина (загатовка ...",False
98,"Инструкция к детской игре должна предполагать,...",False
99,"Покупали шар головоломку, намного интереснее и...",False


Следующая ячейка предназначена для сбора дополнительных данных. 
Может повлиять на производительность и точность модели, так как данные не относятся напрямую к поставленной задаче (отзывы на видеоигры, а не настольные)

In [ ]:
# Считываем дополнительные данные (необязательная ячейка)
extra_reviews = pd.read_csv(r"data\extra_reviews.csv", index_col=0)
extra_reviews

# Объединение с прошлым датасетом
data = pd.concat([data, extra_reviews]).reset_index(drop=True)
data

In [8]:
# Кодируем классы отзывов
enc = LabelEncoder()
label = enc.fit_transform(data["assessment"])
label

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [9]:
# Предобработка текста

processed = data["review"].str.lower()
processed


nltk.download("stopwords")
stop_words = set(stopwords.words("russian"))


processed = processed.apply(
    lambda x: " ".join(term for term in x.split() if term not in stop_words)
)

ps = nltk.PorterStemmer()
processed = processed.apply(lambda x: " ".join(ps.stem(term) for term in x.split()))

tokenizer = RegexpTokenizer(r"\w+")
processed = processed.apply(lambda x: " ".join(tokenizer.tokenize(x)))
processed

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vlads\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0      игра которой победить злодей отличное карточно...
1      обалденная игра компании помогает разнообразит...
2      весёлая настольная игра компании которая раскр...
3      общие впечатления игра огонь взрослого питание...
4      случайно наткнулся игрушку детском мире учитыв...
                             ...                        
96     ерунда полная половина больших бакуганов транс...
97     ужасное качество фишек древесина загатовка так...
98     инструкция детской игре должна предполагать пе...
99     покупали шар головоломку намного интереснее ди...
100    90 00 были мало того сломала пока пыталась пон...
Name: review, Length: 101, dtype: object

In [10]:
# Создаем словарь с наиболее часто встречаемыми словами

nltk.download("punkt_tab")
all_words = []

for text in processed:
    words = word_tokenize(text)
    for w in words:
        all_words.append(w)

all_words = nltk.FreqDist(all_words)

# Print the result
print("Number of words: {}".format(len(all_words)))
print("Most common words: {}".format(all_words.most_common(15)))
word_features = [x[0] for x in all_words.most_common(2000)]

Number of words: 2316
Most common words: [('игра', 78), ('очень', 41), ('игры', 37), ('это', 32), ('достоинства', 21), ('играть', 21), ('недостатки', 20), ('правила', 18), ('игру', 17), ('просто', 16), ('интересная', 14), ('вообще', 14), ('качество', 13), ('2', 12), ('стоит', 12)]


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vlads\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [11]:
# Функция для нахождения фич в тексте
def find_features(text):
    words = word_tokenize(text)
    features = {}
    for word in word_features:
        features[word] = word in words

    return features

In [12]:
# Получаем данные для тренировки (75%) и тестов (25%)
reviews = list(zip(processed, label))

np.random.shuffle(reviews)

feature_set = [(find_features(text), label) for (text, label) in reviews]

training, test = train_test_split(feature_set, test_size=0.25, random_state=1)

print(len(training))
print(len(test))

75
26


In [13]:
# Обучение байесовского классификатора
clf = MultinomialNB()
mulnb_model = SklearnClassifier(clf).train(training)

In [14]:
# Получаем метрики точности
text_features, labels = zip(*test)
prediction = mulnb_model.classify_many(text_features)

print(classification_report(labels, prediction))


pd.DataFrame(
    confusion_matrix(labels, prediction),
    index=[["actual", "actual"], ["positive", "negative"]],
    columns=[["predicted", "predicted"], ["positive", "negative"]],
)

              precision    recall  f1-score   support

           0       0.53      0.90      0.67        10
           1       0.89      0.50      0.64        16

    accuracy                           0.65        26
   macro avg       0.71      0.70      0.65        26
weighted avg       0.75      0.65      0.65        26



predicted         
                 positive negative
actual positive         9        1
       negative         8        8

In [15]:
# Обучение модели логистической регрессии
clf = LogisticRegression(max_iter=100)
logreg_model = SklearnClassifier(clf).train(training)

In [16]:
# Получаем метрики точности для второй модели
text_features, labels = zip(*test)
prediction = logreg_model.classify_many(text_features)

print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index=[["actual", "actual"], ["positive", "negative"]],
    columns=[["predicted", "predicted"], ["positive", "negative"]],
)

              precision    recall  f1-score   support

           0       0.50      0.70      0.58        10
           1       0.75      0.56      0.64        16

    accuracy                           0.62        26
   macro avg       0.62      0.63      0.61        26
weighted avg       0.65      0.62      0.62        26



predicted         
                 positive negative
actual positive         7        3
       negative         7        9

In [17]:
# Сохранение первой модели со словарем фич и функцие нахождения их в тексте
path = r"models\naive_bayes_classifier.pickle"

with open(path, "wb") as classifier_file:
    data_for_save = {
        'model': mulnb_model,
        'features': word_features,
        'function': find_features,
        }
    pickle.dump(data_for_save, classifier_file)

In [17]:
# Сохранение второй модели со словарем фич и функцие нахождения их в тексте
path = r"models\logistic_regression_classifier.pickle"

with open(path, "wb") as classifier_file:
    data_for_save = {
        "model": logreg_model,
        "features": word_features,
        "function": find_features,
    }
    pickle.dump(data_for_save, classifier_file)

{'игра': False, 'очень': False, 'игры': False, 'это': False, 'достоинства': False, 'играть': False, 'недостатки': False, 'правила': False, 'игру': False, 'просто': False, 'интересная': False, 'вообще': False, 'качество': False, '2': False, 'стоит': False, 'то': False, 'рекомендую': False, '3': False, 'интересно': False, 'поле': False, '5': False, 'денег': False, 'всем': False, 'ещё': False, 'нужно': False, 'лет': False, 'яркие': False, 'время': False, 'играем': False, 'играли': False, 'игре': False, 'сама': False, 'коробке': False, 'карты': False, 'быстро': False, 'компании': False, 'карточки': False, 'её': False, '6': False, 'взрослым': False, 'пока': False, 'процесс': False, 'набор': False, 'всё': False, 'такие': False, 'игр': False, 'простые': False, 'большой': False, 'раза': False, 'понравилась': False, 'игрушка': False, 'детям': False, 'развивает': False, 'минут': False, '4': False, 'человек': False, 'удовольствием': False, 'коробка': False, 'цена': False, 'игроков': False, 'инстр